In [1]:
import os
import sys
#os.environ["PYSPARK_PYTHON"] = "/opt/continuum/anaconda/bin/python"
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk1.8.0_221.jdk/Contents/Home/jre"
os.environ["SPARK_HOME"] = "/Users/lcx/package/spark-2.4.6-bin-hadoop2.7"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"

sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [2]:
from pyspark.sql import SparkSession

# create a SparkSession
spark = SparkSession.builder.master("local").appName("twitter ML").getOrCreate()
        #.config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.6.3")\
        #.config("spark.kryoserializer.buffer.max", "1000M")\
        

# get the spark context
sc = spark.sparkContext

In [3]:
spark

### 1. Read data from asw folder, text file

In [4]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [5]:
files_path = "../twitter_data/sample_data/2020_10_30/*/*"

In [6]:
file_schema = StructType([StructField("tweet_text", StringType(), True), 
                          # StructField("hash_tag", ArrayType(StringType(), True), True), 
                          StructField("hash_tag", StringType(), True), 
                          StructField("created_at", StringType(), True), 
                          StructField("retweet_count", IntegerType(), True), 
                          StructField("favorite_count", IntegerType(), True), 
                          StructField("retweeted", BooleanType(), True), 
                          StructField("truncated", BooleanType(), True), 
                          StructField("id", StringType(), True), 
                          StructField("user_name", StringType(), True), 
                          StructField("screen_name", StringType(), True), 
                          StructField("followers_count", IntegerType(), True), 
                          StructField("location", StringType(), True), 
                          StructField("geo", StringType(), True),
                          StructField("invalid", StringType(), True)])

# file_schema = StructType([StructField("_c0", StringType(), True), 
#                           StructField("_c1", ArrayType(StringType(), True), True), 
#                           StructField("_c2", StringType(), True), 
#                           StructField("_c3", IntegerType(), True), 
#                           StructField("_c4", IntegerType(), True), 
#                           StructField("_c5", BooleanType(), True), 
#                           StructField("_c6", BooleanType(), True), 
#                           StructField("_c7", StringType(), True), 
#                           StructField("_c8", StringType(), True), 
#                           StructField("_c9", StringType(), True), 
#                           StructField("_c10", IntegerType(), True), 
#                           StructField("_c11", StringType(), True), 
#                           StructField("_c12", StringType(), True),
#                           StructField("_c13", StringType(), True)])

In [7]:
dir(file_schema)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_needConversion',
 '_needSerializeAnyField',
 'add',
 'fieldNames',
 'fields',
 'fromInternal',
 'fromJson',
 'json',
 'jsonValue',
 'names',
 'needConversion',
 'simpleString',
 'toInternal',
 'typeName']

In [8]:
len(file_schema.fieldNames())

14

In [9]:
file_schema

StructType(List(StructField(tweet_text,StringType,true),StructField(hash_tag,StringType,true),StructField(created_at,StringType,true),StructField(retweet_count,IntegerType,true),StructField(favorite_count,IntegerType,true),StructField(retweeted,BooleanType,true),StructField(truncated,BooleanType,true),StructField(id,StringType,true),StructField(user_name,StringType,true),StructField(screen_name,StringType,true),StructField(followers_count,IntegerType,true),StructField(location,StringType,true),StructField(geo,StringType,true),StructField(invalid,StringType,true)))

In [7]:
# df = (spark
#       .read
#       .csv(header=False, sep="\t", schema=file_schema, enforceSchema=True, 
#            path=files_path)) # don't know why this does not work

df = (spark
      .read
      .format("csv")
      .options(header=False, sep="\t", enforceSchema=True)
      .schema(file_schema)
      .load(files_path))

In [8]:
df.show(3)

+--------------------+--------------------+--------------------+-------------+--------------+---------+---------+-------------------+--------------------+---------------+---------------+--------------------+----+-------+
|          tweet_text|            hash_tag|          created_at|retweet_count|favorite_count|retweeted|truncated|                 id|           user_name|    screen_name|followers_count|            location| geo|invalid|
+--------------------+--------------------+--------------------+-------------+--------------+---------+---------+-------------------+--------------------+---------------+---------------+--------------------+----+-------+
|.@mit_hst + @MITd...|Biology, COVID19,...|Fri Oct 30 20:19:...|            0|             0|    false|     true|1322271849028456448|                 HST|        mit_hst|           2449|       Cambridge, MA|None|   null|
|Just when I am ab...|Past3amSquad, AI,...|Fri Oct 30 20:19:...|            0|             0|    false|     true|132

In [9]:
## write to parquet
parquet_file_path = "/Users/lcx/Documents/weclouddata/my_project/twitter_data/sample_data/2020_10_30_parquet"
(df.write.format("parquet").mode("overwrite").option("compression", "snappy").save(parquet_file_path))

In [10]:
df_sample = df.sample(fraction=0.2, seed=1234)
df_sample = df_sample.coalesce(1)

parquet_file_path = "/Users/lcx/Documents/weclouddata/my_project/twitter_data/sample_data/2020_10_30_subsample_parquet"
(df_sample.write.format("parquet").mode("overwrite").option("compression", "snappy").save(parquet_file_path))

In [9]:
df.printSchema()

root
 |-- tweet_text: string (nullable = true)
 |-- hash_tag: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- retweet_count: integer (nullable = true)
 |-- favorite_count: integer (nullable = true)
 |-- retweeted: boolean (nullable = true)
 |-- truncated: boolean (nullable = true)
 |-- id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- screen_name: string (nullable = true)
 |-- followers_count: integer (nullable = true)
 |-- location: string (nullable = true)
 |-- geo: string (nullable = true)
 |-- invalid: string (nullable = true)



In [10]:
df.take(1)

[Row(tweet_text='.@mit_hst + @MITdeptofBE prof. James Collins spoke about harnessing synthetic #Biology to develop diagnostics for #COVID19 + how his lab is using #DeepLearning  to enhance the design of such systems  @MITEECS #jclinic @MIT_IMES #MachineLearning https://t.co/2u3JwNj7CR https://t.co/EoAMQo7UHx', hash_tag='Biology, COVID19, DeepLearning, jclinic, MachineLearning', created_at='Fri Oct 30 20:19:10 +0000 2020', retweet_count=0, favorite_count=0, retweeted=False, truncated=True, id='1322271849028456448', user_name='HST', screen_name='mit_hst', followers_count=2449, location='Cambridge, MA', geo='None', invalid=None)]

In [14]:
df.count()

35463

In [11]:
print(file_schema.fieldNames())

['tweet_text', 'hash_tag', 'created_at', 'retweet_count', 'favorite_count', 'retweeted', 'truncated', 'id', 'user_name', 'screen_name', 'followers_count', 'location', 'geo', 'invalid']


In [9]:
# columns of interest
cols_select = ['tweet_text', 'hash_tag', 'created_at', 'retweet_count', 'favorite_count']

In [10]:
df_select = df.dropna(subset=["tweet_text"]).select(cols_select)
df_select.show(5)

+--------------------+--------------------+--------------------+-------------+--------------+
|          tweet_text|            hash_tag|          created_at|retweet_count|favorite_count|
+--------------------+--------------------+--------------------+-------------+--------------+
|.@mit_hst + @MITd...|Biology, COVID19,...|Fri Oct 30 20:19:...|            0|             0|
|Just when I am ab...|Past3amSquad, AI,...|Fri Oct 30 20:19:...|            0|             0|
|And a beneficial ...|     AI, FridayReads|Fri Oct 30 20:19:...|            0|             0|
|Check out new wor...|                  AI|Fri Oct 30 20:19:...|            0|             0|
|South Korea 🇰🇷 ...|Healthcare, IoT, ...|Fri Oct 30 20:19:...|            0|             0|
+--------------------+--------------------+--------------------+-------------+--------------+
only showing top 5 rows



In [11]:
df_select.cache()
df_select.count()

35461

In [44]:
# df_select.unpersist()

DataFrame[tweet_text: string, hash_tag: string, created_at: string, retweet_count: int, favorite_count: int]

### 2. Text Processing

#### 2.1. Basic cleaning
1. remove mention @xxx, hashtag #xxx;
2. remove hyperlink;
3. remove special characters;
4. remove any whitespace characters (change it to one)

TODO: </br>
* also fix the abbreviation

In [12]:
df_select_clean = (df_select.withColumn("tweet_text", F.regexp_replace("tweet_text", r"[@#&][A-Za-z0-9_-]+", " "))
                   .withColumn("tweet_text", F.regexp_replace("tweet_text", r"\w+:\/\/\S+", " "))
                   .withColumn("tweet_text", F.regexp_replace("tweet_text", r"[^A-Za-z]", " "))
                   .withColumn("tweet_text", F.regexp_replace("tweet_text", r"\s+", " "))
                   .withColumn("tweet_text", F.lower(F.col("tweet_text")))
                   .withColumn("tweet_text", F.trim(F.col("tweet_text")))
                  )

In [17]:
df_select_clean.show(5)

+--------------------+--------------------+--------------------+-------------+--------------+
|          tweet_text|            hash_tag|          created_at|retweet_count|favorite_count|
+--------------------+--------------------+--------------------+-------------+--------------+
|prof james collin...|Biology, COVID19,...|Fri Oct 30 20:19:...|            0|             0|
|just when i am ab...|Past3amSquad, AI,...|Fri Oct 30 20:19:...|            0|             0|
|and a beneficial ...|     AI, FridayReads|Fri Oct 30 20:19:...|            0|             0|
|check out new wor...|                  AI|Fri Oct 30 20:19:...|            0|             0|
|south korea bus s...|Healthcare, IoT, ...|Fri Oct 30 20:19:...|            0|             0|
+--------------------+--------------------+--------------------+-------------+--------------+
only showing top 5 rows



In [13]:
df_select_clean.cache()

DataFrame[tweet_text: string, hash_tag: string, created_at: string, retweet_count: int, favorite_count: int]

In [18]:
df_sample_pd_unclean = df_select.limit(10).toPandas()

for i in range(10):
    print(df_sample_pd_unclean.iloc[i, 0], "\n")

.@mit_hst + @MITdeptofBE prof. James Collins spoke about harnessing synthetic #Biology to develop diagnostics for #COVID19 + how his lab is using #DeepLearning  to enhance the design of such systems  @MITEECS #jclinic @MIT_IMES #MachineLearning https://t.co/2u3JwNj7CR https://t.co/EoAMQo7UHx 

Just when I am about to have a glass of Gin and lemonade.... #Past3amSquad #AI #MachineLearning #DeepLearning #Coding https://t.co/sAOiSemArA 

And a beneficial use of #AI: Using Artificial Intelligence to Identify Powerful New Antibiotic #FridayReads  https://t.co/K8mPDdPShd 

Check out new work on my @Behance profile: "Community Bank / CREDIT CARD DESIGN" https://t.co/xAF0gSaniv   #AI 

South Korea 🇰🇷 bus 🚌 stops check your temperature 🤒 to fight coronavirus 😷 @wef   #Healthcare #IoT #AI #DigitalTransformation #Technology #Innovation #HealthTech via @automeme #SmartCity #COVID19 cc @mvollmer1 @IrmaRaste @Ronald_vanLoon @ipfconline1  https://t.co/YyW9hmONVX 

ARRIA is the language of data &amp; 

In [19]:
df_sample_pd_clean = df_select_clean.limit(10).toPandas()

In [20]:
for i in range(10):
    print(df_sample_pd_clean.iloc[i, 0], "\n")

prof james collins spoke about harnessing synthetic to develop diagnostics for how his lab is using to enhance the design of such systems 

just when i am about to have a glass of gin and lemonade 

and a beneficial use of using artificial intelligence to identify powerful new antibiotic 

check out new work on my profile community bank credit card design 

south korea bus stops check your temperature to fight coronavirus via cc 

arria is the language of data voice reports reports from join the leaders love always defeats fear 

the latest thinking different daily thanks to 

thank you for featuring our work in beautifully written fourier neural operator gets 1000x speedup on navier stokes pdes as a bonus also talks about time 

reminder massachusetts paid family and medical leave next steps 

labor employment e note october 2020 



In [23]:
df_select_clean.dropna(subset=["tweet_text"]).count()

35461

#### 2.2. Tokenize

In [14]:
from pyspark.ml.feature import Tokenizer

In [15]:
tokenzier = Tokenizer(inputCol="tweet_text", outputCol="tokens")

df_tokenized = tokenzier.transform(df_select_clean)

In [26]:
df_tokenized.show(5)

+--------------------+--------------------+--------------------+-------------+--------------+--------------------+
|          tweet_text|            hash_tag|          created_at|retweet_count|favorite_count|              tokens|
+--------------------+--------------------+--------------------+-------------+--------------+--------------------+
|prof james collin...|Biology, COVID19,...|Fri Oct 30 20:19:...|            0|             0|[prof, james, col...|
|just when i am ab...|Past3amSquad, AI,...|Fri Oct 30 20:19:...|            0|             0|[just, when, i, a...|
|and a beneficial ...|     AI, FridayReads|Fri Oct 30 20:19:...|            0|             0|[and, a, benefici...|
|check out new wor...|                  AI|Fri Oct 30 20:19:...|            0|             0|[check, out, new,...|
|south korea bus s...|Healthcare, IoT, ...|Fri Oct 30 20:19:...|            0|             0|[south, korea, bu...|
+--------------------+--------------------+--------------------+-------------+--

**some quick analyses**

In [30]:
df_select_clean_analysis = df_tokenized.withColumn("wordCounts", F.size(F.col("tokens")))
df_select_clean_analysis.show(5)

+--------------------+--------------------+--------------------+-------------+--------------+--------------------+----------+
|          tweet_text|            hash_tag|          created_at|retweet_count|favorite_count|              tokens|wordCounts|
+--------------------+--------------------+--------------------+-------------+--------------+--------------------+----------+
|prof james collin...|Biology, COVID19,...|Fri Oct 30 20:19:...|            0|             0|[prof, james, col...|        23|
|just when i am ab...|Past3amSquad, AI,...|Fri Oct 30 20:19:...|            0|             0|[just, when, i, a...|        13|
|and a beneficial ...|     AI, FridayReads|Fri Oct 30 20:19:...|            0|             0|[and, a, benefici...|        13|
|check out new wor...|                  AI|Fri Oct 30 20:19:...|            0|             0|[check, out, new,...|        12|
|south korea bus s...|Healthcare, IoT, ...|Fri Oct 30 20:19:...|            0|             0|[south, korea, bu...|    

In [40]:
agg_col = ["wordCounts", "retweet_count", "favorite_count"]
# [F.min(F.col(c)).alias(c+"_min") for c in agg_col]
df_select_clean_analysis.agg(*[F.mean(F.col(c)).alias(c+"_mean") for c in agg_col],
                             *[F.min(F.col(c)).alias(c+"_min") for c in agg_col], 
                             *[F.max(F.col(c)).alias(c+"_max") for c in agg_col]).show()

+-----------------+------------------+-------------------+--------------+-----------------+------------------+--------------+-----------------+------------------+
|  wordCounts_mean|retweet_count_mean|favorite_count_mean|wordCounts_min|retweet_count_min|favorite_count_min|wordCounts_max|retweet_count_max|favorite_count_max|
+-----------------+------------------+-------------------+--------------+-----------------+------------------+--------------+-----------------+------------------+
|14.99416260116748|               0.0|                0.0|             1|                0|                 0|            61|                0|                 0|
+-----------------+------------------+-------------------+--------------+-----------------+------------------+--------------+-----------------+------------------+



In [39]:
df_select_clean_analysis.select(F.avg(F.col("wordCounts")), 
                                F.min(F.col("wordCounts")), 
                                F.max(F.col("wordCounts"))).show()

+-----------------+---------------+---------------+
|  avg(wordCounts)|min(wordCounts)|max(wordCounts)|
+-----------------+---------------+---------------+
|14.99416260116748|              1|             61|
+-----------------+---------------+---------------+



In [48]:
df_select_clean_analysis.approxQuantile(col="wordCounts", 
                                        probabilities=[0.25, 0.5, 0.75], 
                                        relativeError=0.1)

[8.0, 13.0, 20.0]

#### 2.3. Remove stop words
TODO: extend stop words </br>
("also", "via", "cc", "rt", "must", "always"...)

In [16]:
from pyspark.ml.feature import StopWordsRemover

In [17]:
stopword_remover = StopWordsRemover(inputCol="tokens", outputCol="remove_stop")

In [18]:
stopwords_list = stopword_remover.getStopWords()
print(len(stopwords_list))
print(stopwords_list[:10])
# print(sorted(stopword_remover.getStopWords()))

181
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']


In [19]:
more_stopwords = ["also", "via", "cc", "rt", "must", "always"]
stopwords_list = stopwords_list + more_stopwords
print(stopwords_list[-10:])

["where's", "who's", "why's", 'would', 'also', 'via', 'cc', 'rt', 'must', 'always']


In [20]:
stopword_remover.setStopWords(stopwords_list)
print(stopword_remover.getStopWords())

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [21]:
df_rmstop = stopword_remover.transform(df_tokenized)

In [22]:
df_rmstop.show(5)

+--------------------+--------------------+--------------------+-------------+--------------+--------------------+--------------------+
|          tweet_text|            hash_tag|          created_at|retweet_count|favorite_count|              tokens|         remove_stop|
+--------------------+--------------------+--------------------+-------------+--------------+--------------------+--------------------+
|prof james collin...|Biology, COVID19,...|Fri Oct 30 20:19:...|            0|             0|[prof, james, col...|[prof, james, col...|
|just when i am ab...|Past3amSquad, AI,...|Fri Oct 30 20:19:...|            0|             0|[just, when, i, a...|[glass, gin, lemo...|
|and a beneficial ...|     AI, FridayReads|Fri Oct 30 20:19:...|            0|             0|[and, a, benefici...|[beneficial, use,...|
|check out new wor...|                  AI|Fri Oct 30 20:19:...|            0|             0|[check, out, new,...|[check, new, work...|
|south korea bus s...|Healthcare, IoT, ...|Fri O

In [58]:
# df_sample_pd_before = df_tokenized.limit(10).toPandas()
df_sample_pd_rmstop = df_rmstop.limit(10).toPandas()

for i in range(10):
    print(df_sample_pd_rmstop.loc[i, "tokens"], "\n")
    print(df_sample_pd_rmstop.loc[i, "remove_stop"], "\n")

['prof', 'james', 'collins', 'spoke', 'about', 'harnessing', 'synthetic', 'to', 'develop', 'diagnostics', 'for', 'how', 'his', 'lab', 'is', 'using', 'to', 'enhance', 'the', 'design', 'of', 'such', 'systems'] 

['prof', 'james', 'collins', 'spoke', 'harnessing', 'synthetic', 'develop', 'diagnostics', 'lab', 'using', 'enhance', 'design', 'systems'] 

['just', 'when', 'i', 'am', 'about', 'to', 'have', 'a', 'glass', 'of', 'gin', 'and', 'lemonade'] 

['glass', 'gin', 'lemonade'] 

['and', 'a', 'beneficial', 'use', 'of', 'using', 'artificial', 'intelligence', 'to', 'identify', 'powerful', 'new', 'antibiotic'] 

['beneficial', 'use', 'using', 'artificial', 'intelligence', 'identify', 'powerful', 'new', 'antibiotic'] 

['check', 'out', 'new', 'work', 'on', 'my', 'profile', 'community', 'bank', 'credit', 'card', 'design'] 

['check', 'new', 'work', 'profile', 'community', 'bank', 'credit', 'card', 'design'] 

['south', 'korea', 'bus', 'stops', 'check', 'your', 'temperature', 'to', 'fight', 'cor

#### 2.4. Lemmatization
**Note:** </br>
1. here we use the stemming for fast processing;
2. One may use the lemmatization in future. Note that one should define a parts-of-speech to obtain the correct lemma

In [28]:
df_rmstop.printSchema()

root
 |-- tweet_text: string (nullable = true)
 |-- hash_tag: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- retweet_count: integer (nullable = true)
 |-- favorite_count: integer (nullable = true)
 |-- tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- remove_stop: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [23]:
# Import stemmer library
from nltk.stem.porter import PorterStemmer

In [24]:
# Instantiate stemmer object
stemmer = PorterStemmer()

**so pandas_udf cannot handle complex data?**

In [25]:
def stem(words_list):
    return [stemmer.stem(word) for word in words_list]

stem_udf = F.udf(stem, returnType = ArrayType(StringType()))

In [26]:
df_stemmed = df_rmstop.select("remove_stop", stem_udf(F.col("remove_stop")).alias("stemmed")).limit(10).toPandas()

In [27]:
for i in range(10):
    print(df_stemmed.loc[i, "remove_stop"], "\n")
    print(df_stemmed.loc[i, "stemmed"], "\n")

['prof', 'james', 'collins', 'spoke', 'harnessing', 'synthetic', 'develop', 'diagnostics', 'lab', 'using', 'enhance', 'design', 'systems'] 

['prof', 'jame', 'collin', 'spoke', 'har', 'synthet', 'develop', 'diagnost', 'lab', 'use', 'enhanc', 'design', 'system'] 

['glass', 'gin', 'lemonade'] 

['glass', 'gin', 'lemonad'] 

['beneficial', 'use', 'using', 'artificial', 'intelligence', 'identify', 'powerful', 'new', 'antibiotic'] 

['benefici', 'use', 'use', 'artifici', 'intellig', 'identifi', 'power', 'new', 'antibiot'] 

['check', 'new', 'work', 'profile', 'community', 'bank', 'credit', 'card', 'design'] 

['check', 'new', 'work', 'profil', 'commun', 'bank', 'credit', 'card', 'design'] 

['south', 'korea', 'bus', 'stops', 'check', 'temperature', 'fight', 'coronavirus'] 

['south', 'korea', 'bu', 'stop', 'check', 'temperatur', 'fight', 'coronaviru'] 

['arria', 'language', 'data', 'voice', 'reports', 'reports', 'join', 'leaders', 'love', 'defeats', 'fear'] 

['arria', 'languag', 'data', 

In [29]:
df_stemmed = df_rmstop.withColumn("stemmed", stem_udf(F.col("remove_stop")))

df_stemmed.show(5)

+--------------------+--------------------+--------------------+-------------+--------------+--------------------+--------------------+--------------------+
|          tweet_text|            hash_tag|          created_at|retweet_count|favorite_count|              tokens|         remove_stop|             stemmed|
+--------------------+--------------------+--------------------+-------------+--------------+--------------------+--------------------+--------------------+
|prof james collin...|Biology, COVID19,...|Fri Oct 30 20:19:...|            0|             0|[prof, james, col...|[prof, james, col...|[prof, jame, coll...|
|just when i am ab...|Past3amSquad, AI,...|Fri Oct 30 20:19:...|            0|             0|[just, when, i, a...|[glass, gin, lemo...|[glass, gin, lemo...|
|and a beneficial ...|     AI, FridayReads|Fri Oct 30 20:19:...|            0|             0|[and, a, benefici...|[beneficial, use,...|[benefici, use, u...|
|check out new wor...|                  AI|Fri Oct 30 20:1

In [30]:
df_stemmed.cache()
df_stemmed.count()

35461

#### 2.5. More to consider
1. use bigram
2. filter short vector

#### 2.6. Encode words

In [31]:
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF

##### 2.6.1. CountVectorizer

In [32]:
# should tune some parameter
vectorizer = CountVectorizer(inputCol= "stemmed", outputCol="rawFeatures")
vectorizer_model = vectorizer.fit(df_stemmed)

In [33]:
df_vect = vectorizer_model.transform(df_stemmed)

In [34]:
df_vect.select("stemmed", "rawFeatures").show(5, truncate=False)

+----------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+
|stemmed                                                                                       |rawFeatures                                                                                                          |
+----------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+
|[prof, jame, collin, spoke, har, synthet, develop, diagnost, lab, use, enhanc, design, system]|(11147,[3,17,36,123,362,651,928,1157,1310,1621,1638,2268,4047],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|[glass, gin, lemonad]                                                                         |(11147,[2311,6242,6464],[1.0,1.0,1.0])      

##### 2.6.2. TF-IDF
**do we need IDF for the LDA model?**

In [35]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf_model = idf.fit(df_vect)

In [36]:
df_idf = idf_model.transform(df_vect)

In [37]:
df_idf.select("rawFeatures", "features").show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|rawFeatures                                                                                                          |features                                                                                                                                                                                                                                                                                                   |
+---------------------------------------------------------------------------------------------------------------------+-----------------------------------------

#### 2.7. Fit a LDA model

In [38]:
from pyspark.ml.clustering import LDA

In [39]:
lda = LDA(k=10, seed=1234, optimizer="em")

In [40]:
lda_mode = lda.fit(df_idf)

In [91]:
type(lda_mode)

pyspark.ml.clustering.DistributedLDAModel

In [41]:
topics_describe = lda_mode.describeTopics()
topics_describe.show(truncate=False)

+-----+---------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|topic|termIndices                                  |termWeights                                                                                                                                                                                                                    |
+-----+---------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |[59, 48, 29, 36, 25, 53, 222, 109, 134, 215] |[0.011947539463108494, 0.01063126480549928, 0.010105796113778636, 0.008882373253693154, 0.00837318339170578, 0.00

##### 2.7.1. convert the word vector to vocabulary

In [42]:
vectorizer_model.getVocabSize()

262144

In [43]:
vectorizer_model.vocabulary[0:10]

['learn',
 'ai',
 'data',
 'use',
 'intellig',
 'machin',
 'artifici',
 'new',
 'read',
 'help']

In [44]:
def termsIdx2Term(vocabulary):
    def termsIdx2Term(termIndices):
        return [vocabulary[int(index)] for index in termIndices]
    return F.udf(termsIdx2Term, ArrayType(StringType()))

vocabList = vectorizer_model.vocabulary

In [45]:
final = topics_describe.withColumn("Terms", termsIdx2Term(vocabList)("termIndices"))

In [46]:
final.select("Terms").show(truncate=False)

+--------------------------------------------------------------------------------+
|Terms                                                                           |
+--------------------------------------------------------------------------------+
|[digit, process, compani, system, free, report, trust, secur, video, play]      |
|[technolog, time, global, code, day, busi, program, tutori, use, artifici]      |
|[learn, machin, custom, cloud, success, integr, like, insight, pm, cool]        |
|[take, iot, innov, talk, watch, see, patient, healthcar, like, decis]           |
|[learn, covid, record, best, machin, cough, model, avail, asymptomat, pleas]    |
|[learn, ai, best, trend, make, platform, build, get, start, machin]             |
|[learn, research, scienc, machin, artifici, intellig, data, new, network, visit]|
|[ai, predict, get, billion, develop, know, built, learn, mobil, onlin]          |
|[valu, analyt, gener, help, product, use, smart, across, intellig, boost]       |
|[da

##### 2.7.2. check the model performance
**Note:** the calculation of log Perplexity requires a large topicsMatrix() to the driver. So one should use subsample...

In [96]:
lda_mode.logPerplexity(df_idf)

11.142064829812528

In [47]:
df_sample_check_score = df_idf.sample(fraction=0.2, seed=1234)

In [48]:
lda_mode.logPerplexity(df_sample_check_score)

23.7477518265845

In [112]:
# spark.catalog.clearCache()

In [61]:
lda_mode.transform(df_sample_check_score).select("tweet_text", 
                                                 "topicDistribution").show(5, truncate=False)

+----------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tweet_text                                                                                    |topicDistribution                                                                                                                                                                                        |
+----------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|the latest thinking different daily thanks to                                                 |[0.0933

In [143]:
lda_mode.trainingLogLikelihood()

-12407824.177683208

##### 2.7.3. let's try not using IDF (bad idea)
**Note:** Very bad. The topics will be dominated by those frequent words, machine learning, artificial intellegence

In [49]:
lda_tf = LDA(k=10, seed=1234, optimizer="em", featuresCol="rawFeatures")

lda_tf_model = lda_tf.fit(df_vect)

In [50]:
topics_describe_tf = lda_tf_model.describeTopics()
topics_describe_tf.show(truncate=False)

+-----+-------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|topic|termIndices                    |termWeights                                                                                                                                                                                                                 |
+-----+-------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |[0, 1, 2, 4, 3, 5, 6, 7, 8, 10]|[0.016355966530646766, 0.01368392013664132, 0.01078462281003215, 0.009592814521427417, 0.009523470500275499, 0.00897987390714028, 0.00833056408119102, 0.006137820622040562, 0.005

In [51]:
final_tf = topics_describe_tf.withColumn("Terms", termsIdx2Term(vocabList)("termIndices"))

In [52]:
final_tf.select("Terms").show(truncate=False)

+-------------------------------------------------------------------+
|Terms                                                              |
+-------------------------------------------------------------------+
|[learn, ai, data, intellig, use, machin, artifici, new, read, busi]|
|[learn, ai, data, intellig, use, machin, artifici, new, read, help]|
|[learn, ai, data, intellig, machin, use, artifici, new, read, help]|
|[learn, ai, data, use, intellig, machin, artifici, new, read, help]|
|[learn, ai, data, use, machin, intellig, artifici, new, read, help]|
|[learn, ai, data, use, machin, intellig, artifici, new, read, help]|
|[learn, ai, data, use, intellig, machin, artifici, new, read, help]|
|[learn, ai, data, use, intellig, machin, artifici, new, read, busi]|
|[learn, ai, data, use, intellig, machin, artifici, new, read, help]|
|[learn, ai, data, intellig, use, machin, artifici, read, new, help]|
+-------------------------------------------------------------------+



In [56]:
df_vect_sample_check_score = df_vect.sample(fraction=0.2, seed=1234)
lda_tf_model.logPerplexity(df_vect_sample_check_score)

15.440127189501274

In [144]:
lda_tf_model.trainingLogLikelihood()

-2348721.882527943

#### 3. Check the topic distribution

In [64]:
cluster_df_sample = lda_mode.transform(df_sample_check_score).select("tweet_text", "topicDistribution")

In [71]:
cluster_df_sample.select("topicDistribution").printSchema()

root
 |-- topicDistribution: vector (nullable = true)



In [72]:
cluster_df_sample.select("topicDistribution").limit(2).rdd.reduce(lambda x,y: x.toArray() + y.toArray())

[Row(topicDistribution=DenseVector([0.0934, 0.0968, 0.1134, 0.104, 0.1029, 0.0864, 0.1062, 0.0917, 0.0903, 0.1149])),
 Row(topicDistribution=DenseVector([0.1621, 0.0703, 0.0672, 0.0666, 0.1206, 0.0686, 0.1824, 0.1196, 0.0763, 0.0663]))]

In [76]:
from operator import add
cluster_df_sample.select("topicDistribution").limit(2).rdd.reduce(lambda x,y: x.toArray() + y.toArray())

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1770.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1770.0 (TID 880, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/lcx/package/spark-2.4.6-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/sql/types.py", line 1533, in __getattr__
    idx = self.__fields__.index(item)
ValueError: 'toArray' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/lcx/package/spark-2.4.6-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/Users/lcx/package/spark-2.4.6-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  F

In [77]:
to_array = F.udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

In [85]:
cluster_df_sample_toArray = cluster_df_sample.withColumn("topicDistributionArray", 
                                                         to_array("topicDistribution"))

In [90]:
cluster_df_sample_overall = cluster_df_sample_toArray.agg(
    F.array(*[F.sum(F.col("topicDistributionArray")[i]) for i in range(10)]).alias("topics_Overall"))

cluster_df_sample_overall.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|topics_Overall                                                                                                                                                                              |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[723.6276314426214, 705.2695701345801, 705.854123800993, 696.3106028027833, 714.9629479851574, 715.0801446791738, 708.341060584411, 722.5668835528195, 705.5965143069625, 701.3905211668462]|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+



In [107]:
topics_info = cluster_df_sample_overall.toPandas()

In [113]:
topics_nums = topics_info.loc[0, "topics_Overall"]

In [114]:
import pandas as pd

In [117]:
["topic_" + str(i) for i in range(10)]

['topic_0',
 'topic_1',
 'topic_2',
 'topic_3',
 'topic_4',
 'topic_5',
 'topic_6',
 'topic_7',
 'topic_8',
 'topic_9']

In [130]:
topics_info_final = pd.DataFrame(topics_nums, index= ["topic_" + str(i) for i in range(10)], columns=["count"])

In [133]:
topics_info_final = topics_info_final.astype('int32')

In [134]:
topics_info_final

,count
topic_0,723
topic_1,705
topic_2,705
topic_3,696
topic_4,714
topic_5,715
topic_6,708
topic_7,722
topic_8,705
topic_9,701


In [136]:
cluster_df_sample_toArray.show(4)

+--------------------+--------------------+----------------------+
|          tweet_text|   topicDistribution|topicDistributionArray|
+--------------------+--------------------+----------------------+
|the latest thinki...|[0.09336869368277...|  [0.093368694, 0.0...|
|weekly covid over...|[0.16212564426412...|  [0.16212565, 0.07...|
|a pytorch re impl...|[0.07550098057259...|  [0.07550098, 0.08...|
|beyond the hype a...|[0.06360133525585...|  [0.06360134, 0.31...|
+--------------------+--------------------+----------------------+
only showing top 4 rows



In [140]:
cluster_df_sample_splits = cluster_df_sample_toArray.select(
                                                            [(F.col("topicDistributionArray")[i]).alias("topic_"+str(i)) for i in range(10)])

In [141]:
cluster_df_sample_splits.show(5)

+-----------+----------+-----------+-----------+----------+-----------+-----------+-----------+----------+-----------+
|    topic_0|   topic_1|    topic_2|    topic_3|   topic_4|    topic_5|    topic_6|    topic_7|   topic_8|    topic_9|
+-----------+----------+-----------+-----------+----------+-----------+-----------+-----------+----------+-----------+
|0.093368694|0.09681352| 0.11344034|0.103976846|0.10290956| 0.08637614| 0.10619131|  0.0917152|0.09031064| 0.11489776|
| 0.16212565|0.07032451|0.067157984| 0.06660924|0.12064729| 0.06856954| 0.18237531| 0.11959137|0.07634403| 0.06625508|
| 0.07550098|0.08948835| 0.18738498| 0.09091354|0.08460948|  0.0824092| 0.07731093|0.087777644|0.14683178|0.077773124|
| 0.06360134|0.31425086| 0.06616193|0.078847855| 0.0749395|0.085080855|0.077667326| 0.06145126|0.10746074| 0.07053835|
| 0.28822455|0.10308446|  0.0700133| 0.06900075|0.06809111|0.096229084| 0.06805478| 0.06913701|0.09985898|   0.068306|
+-----------+----------+-----------+-----------+

### 4. put it into a pipeline

In [9]:
# from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml import Pipeline
from pyspark.ml.clustering import LDA
from sparknlp.annotator import Stemmer

In [14]:
# Import stemmer library
from nltk.stem.porter import PorterStemmer

In [15]:
# set seed
seedNum = 1234

# set parameters
more_stopwords = ["also", "via", "cc", "rt", "must", "always"]

n_topics = 10

In [16]:
def termsIdx2Term(vocabulary):
    def termsIdx2Term(termIndices):
        return [vocabulary[int(index)] for index in termIndices]
    return F.udf(termsIdx2Term, ArrayType(StringType()))

In [7]:
# 1. load file
files_path = "../twitter_data/sample_data/2020_10_30/*/*"

file_schema = StructType([StructField("tweet_text", StringType(), True), 
                          # StructField("hash_tag", ArrayType(StringType(), True), True), 
                          StructField("hash_tag", StringType(), True), 
                          StructField("created_at", StringType(), True), 
                          StructField("retweet_count", IntegerType(), True), 
                          StructField("favorite_count", IntegerType(), True), 
                          StructField("retweeted", BooleanType(), True), 
                          StructField("truncated", BooleanType(), True), 
                          StructField("id", StringType(), True), 
                          StructField("user_name", StringType(), True), 
                          StructField("screen_name", StringType(), True), 
                          StructField("followers_count", IntegerType(), True), 
                          StructField("location", StringType(), True), 
                          StructField("geo", StringType(), True),
                          StructField("invalid", StringType(), True)])


# how to prepare the train and test dataset
df = (spark
      .read
      .format("csv")
      .options(header=False, sep="\t", enforceSchema=True)
      .schema(file_schema)
      .load(files_path))

# columns of interest
cols_select = ['tweet_text', 'hash_tag', 'created_at', 'retweet_count', 'favorite_count']

df_select = df.dropna(subset=["tweet_text"]).select(cols_select)

# 2. text processing

df_select_clean = (df_select.withColumn("tweet_text", F.regexp_replace("tweet_text", r"[@#&][A-Za-z0-9_-]+", " "))
                   .withColumn("tweet_text", F.regexp_replace("tweet_text", r"\w+:\/\/\S+", " "))
                   .withColumn("tweet_text", F.regexp_replace("tweet_text", r"[^A-Za-z]", " "))
                   .withColumn("tweet_text", F.regexp_replace("tweet_text", r"\s+", " "))
                   .withColumn("tweet_text", F.lower(F.col("tweet_text")))
                   .withColumn("tweet_text", F.trim(F.col("tweet_text")))
                  )


df_select_clean.cache()
print("total tweets: ", df_select_clean.count())

total tweets:  35461


In [8]:
# train, test = df_select_clean.randomSplit([0.8, 0.2], seed=seedNum)

In [17]:
#============================================
# preprocessing
#============================================
# 2.1. tokenize
tokenizer = Tokenizer(inputCol="tweet_text", outputCol="tokens")

# 2.2. remove stopwords
stopword_remover = StopWordsRemover(inputCol="tokens", outputCol="remove_stop")

stopwords_list = stopword_remover.getStopWords()
stopwords_list = stopwords_list + more_stopwords
stopword_remover.setStopWords(stopwords_list)
#2.3. stemming
# TODO: how to modify the stemming function into a transformer?
stemmer = PorterStemmer()
# more straightforward to use lambda
stem_udf = F.udf(lambda l : [stemmer.stem(word) for word in l], returnType = ArrayType(StringType()))
# To integrate into a pipeline, using spark NLP
# stemmer = Stemmer().setInputCols(["remove_stop"]).setOutputCol("stemmed")

In [18]:
df_tokenized = tokenizer.transform(df_select_clean)
df_rmstop = stopword_remover.transform(df_tokenized)
df_stemmed = df_rmstop.withColumn("stemmed", stem_udf(F.col("remove_stop")))

df_stemmed.cache()
df_select_clean.unpersist()

DataFrame[tweet_text: string, hash_tag: string, created_at: string, retweet_count: int, favorite_count: int]

In [19]:
train, test = df_stemmed.randomSplit([0.8, 0.2], seed=seedNum)

In [14]:
# 2.4. CountVectorizer
vectorizer = CountVectorizer(inputCol= "stemmed", outputCol="rawFeatures")
# 2.5. IDf
idf = IDF(inputCol="rawFeatures", outputCol="features")

# 3. train the LDA model
lda = LDA(k=n_topics, seed=seedNum, optimizer="em")

In [15]:
pipeline = Pipeline(stages=[vectorizer, idf, lda])

In [16]:
pipeline_model = pipeline.fit(train)

In [17]:
pipelinePath = "./LDA-pipeline-model" 

pipeline_model.write().overwrite().save(pipelinePath)

##### 4.1. get the LDA model and other useful part

In [18]:
#
lda_model = pipeline_model.stages[-1]

vectorizer_model = pipeline_model.stages[0]
vocabList = vectorizer_model.vocabulary

In [20]:
# show the topics and the associated words
topics_describe = lda_model.describeTopics()

final = topics_describe.withColumn("Terms", termsIdx2Term(vocabList)("termIndices"))

final.select("Terms").show(truncate=False)

+-----------------------------------------------------------------------------------+
|Terms                                                                              |
+-----------------------------------------------------------------------------------+
|[base, day, cough, detect, record, recommend, asymptomat, role, work, product]     |
|[research, great, organ, iot, talk, social, ethic, p, grow, thing]                 |
|[program, learn, intellig, take, virtual, system, innov, financi, decis, healthcar]|
|[news, tool, top, brain, ai, report, artifici, defeat, voic, fear]                 |
|[ai, data, latest, join, potenti, us, read, creat, report, novemb]                 |
|[learn, billion, challeng, differ, ai, qualiti, machin, tree, artifici, data]      |
|[ai, scienc, get, predict, global, learn, built, price, market, give]              |
|[check, learn, deep, machin, best, vs, one, award, help, track]                    |
|[free, use, app, onlin, visit, look, technolog, stude

#### 4.2. evaluate

In [21]:
lda_model.trainingLogLikelihood()

-9897287.829925386

In [45]:
print(f"LogLikelihood train: {lda_model.trainingLogLikelihood(): .4f}")
print(f"LogLikelihood test:")

LogLikelihood train: -9897287.8299
LogLikelihood test:


In [26]:
lda_model.logLikelihood(idf_model.transform(vectorizer_model.transform(test)))

-7639602.470720407

In [24]:
idf_model = pipeline_model.stages[1]

In [27]:
lda_model.logPerplexity(idf_model.transform(vectorizer_model.transform(train)))

11.532331662921242

In [25]:
lda_model.logPerplexity(idf_model.transform(vectorizer_model.transform(test)))

23.71173770845965

#### 4.3. Extract more information

In [ ]:
[F.sum(F.col("topic_"+str(i) for i in range(n_topicstopics)))]

In [28]:
# 5. check the topic distribution among dataset
df_with_topics = pipeline_model.transform(test).select("tweet_text", "topicDistribution")
to_array = F.udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))
df_with_topics_toArray = df_with_topics.select("tweet_text", to_array("topicDistribution").alias("topicDistributionArray"))

In [ ]:
df_with_topics_toArray.show(5, truncate=False)

In [31]:
df_with_topics_final = df_with_topics_toArray.select(["tweet_text"] + 
                                                     [(F.col("topicDistributionArray")[i]).alias("topic_"+str(i)) for i in range(10)])

In [ ]:
df_with_topics_final.show(5)

In [37]:
df_with_topics_final.agg(*[F.sum(F.col("topic_"+str(i))).alias("topic_"+str(i)+ "_count") for i in range(n_topics)]).show()

+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|    topic_0_count|    topic_1_count|    topic_2_count|    topic_3_count|    topic_4_count|    topic_5_count|    topic_6_count|    topic_7_count|    topic_8_count|    topic_9_count|
+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|710.9265619385988|695.2507085613906|719.5939190723002|703.1157946716994|680.9508484285325|710.3112846724689|692.3221873827279|693.8059616852552|680.4239798728377|705.2987543363124|
+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+



In [38]:
topics_info_final = df_with_topics_final.agg(*[F.sum(F.col("topic_"+str(i)))
                                               .alias("topic_"+str(i)+ "_count") 
                                               for i in range(n_topics)]).toPandas()

In [40]:
topics_info_final.astype('int32').T

,0
topic_0_count,710
topic_1_count,695
topic_2_count,719
topic_3_count,703
topic_4_count,680
topic_5_count,710
topic_6_count,692
topic_7_count,693
topic_8_count,680
topic_9_count,705


### 5. Load the train model

In [23]:
from pyspark.ml import PipelineModel

In [12]:
import json

In [9]:
stopwords_list = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', "i'll", "you'll", "he'll", "she'll", "we'll", "they'll", "i'd", "you'd", "he'd", "she'd", "we'd", "they'd", "i'm", "you're", "he's", "she's", "it's", "we're", "they're", "i've", "we've", "you've", "they've", "isn't", "aren't", "wasn't", "weren't", "haven't", "hasn't", "hadn't", "don't", "doesn't", "didn't", "won't", "wouldn't", "shan't", "shouldn't", "mustn't", "can't", "couldn't", 'cannot', 'could', "here's", "how's", "let's", 'ought', "that's", "there's", "what's", "when's", "where's", "who's", "why's", 'would', 'also', 'via', 'cc', 'rt', 'must', 'always']

In [11]:
print(len(stopwords_list))

187


In [14]:
with open("/Users/lcx/Documents/weclouddata/my_project/spark_ML/stopwords_list.json", "w") as fp:
    json.dump(stopwords_list, fp)

In [15]:
with open("/Users/lcx/Documents/weclouddata/my_project/spark_ML/stopwords_list.json", "r") as fp:
    stopwords_list_load = json.load(fp)

In [17]:
print(len(stopwords_list_load))

187


In [ ]:
extra_for_stemmed = ["learn", "ai", "machin", "use", "intellig", "new", "artifici", 
                     'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
                     'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [ ]:
with open("/Users/lcx/Documents/weclouddata/my_project/spark_ML/extra_for_stemmed.json", "w") as fp:
    json.dump(extra_for_stemmed, fp)

In [21]:
pipelinePath = "/Users/lcx/Documents/weclouddata/my_project/spark_ML/models/LDA-pipeline-model_test"

In [24]:
savedPipelineModel = PipelineModel.load(pipelinePath)

In [34]:
df_with_topics = savedPipelineModel.transform(test).select("tweet_text", "topicDistribution")

to_array = F.udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))
df_with_topics_toArray = df_with_topics.select("tweet_text", to_array("topicDistribution").alias("topicDistributionArray"))

df_with_topics_final = df_with_topics_toArray.select(["tweet_text"] + 
                                                 [(F.col("topicDistributionArray")[i]).alias("topic_"+str(i)) for i in range(10)])

df_topic_info = df_with_topics_final.agg(*[F.floor(F.sum(F.col("topic_"+str(i)))).alias("topic_"+str(i)+ "_count") for i in range(n_topics)])

In [35]:
df_topic_info.cache()
df_topic_info.show()

+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
|topic_0_count|topic_1_count|topic_2_count|topic_3_count|topic_4_count|topic_5_count|topic_6_count|topic_7_count|topic_8_count|topic_9_count|
+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
|          716|          698|          703|          692|          688|          709|          700|          694|          696|          697|
+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+



In [36]:
df_topic_info_count = df_topic_info.withColumn("count", F.lit("count"))

In [42]:
df_topic_info_count.groupby().pivot("count").agg(*[F.first(F.col("topic_"+str(i) + "_count")) for i in range(n_topics)]).show()

+---------------------------------+---------------------------------+---------------------------------+---------------------------------+---------------------------------+---------------------------------+---------------------------------+---------------------------------+---------------------------------+---------------------------------+
|count_first(topic_0_count, false)|count_first(topic_1_count, false)|count_first(topic_2_count, false)|count_first(topic_3_count, false)|count_first(topic_4_count, false)|count_first(topic_5_count, false)|count_first(topic_6_count, false)|count_first(topic_7_count, false)|count_first(topic_8_count, false)|count_first(topic_9_count, false)|
+---------------------------------+---------------------------------+---------------------------------+---------------------------------+---------------------------------+---------------------------------+---------------------------------+---------------------------------+---------------------------------+---------

In [44]:
df_topic_info_count.groupby([F.col("topic_"+str(i) + "_count") for i in range(n_topics)]).pivot("count").sum().show()

+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+------------------------+
|topic_0_count|topic_1_count|topic_2_count|topic_3_count|topic_4_count|topic_5_count|topic_6_count|topic_7_count|topic_8_count|topic_9_count|count_sum(topic_0_count)|count_sum(topic_1_count)|count_sum(topic_2_count)|count_sum(topic_3_count)|count_sum(topic_4_count)|count_sum(topic_5_count)|count_sum(topic_6_count)|count_sum(topic_7_count)|count_sum(topic_8_count)|count_sum(topic_9_count)|
+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+------------------------+------------------------+------------------------+

In [46]:
df_topic_info_count.show()

+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-----+
|topic_0_count|topic_1_count|topic_2_count|topic_3_count|topic_4_count|topic_5_count|topic_6_count|topic_7_count|topic_8_count|topic_9_count|count|
+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-----+
|          716|          698|          703|          692|          688|          709|          700|          694|          696|          697|count|
+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-----+



In [61]:
df_topic_info.selectExpr("stack(10, 'topic_0', topic_0_count, \
                               'topic_1', topic_1_count, \
                               'topic_2', topic_2_count, \
                               'topic_3', topic_3_count, \
                               'topic_4', topic_4_count, \
                               'topic_5', topic_5_count, \
                               'topic_6', topic_6_count, \
                               'topic_7', topic_7_count, \
                               'topic_8', topic_8_count, \
                               'topic_9', topic_9_count)").withColumnRenamed("col0","topic").withColumnRenamed("col1","count").show()

+-------+-----+
|  topic|count|
+-------+-----+
|topic_0|  716|
|topic_1|  698|
|topic_2|  703|
|topic_3|  692|
|topic_4|  688|
|topic_5|  709|
|topic_6|  700|
|topic_7|  694|
|topic_8|  696|
|topic_9|  697|
+-------+-----+



### 6. Train with pypackage

In [7]:
sys.path.insert(0, "/Users/lcx/Documents/weclouddata/my_project/spark_ML/sparkLDA.zip")

In [7]:
sys.path.insert(0, "/Users/lcx/Documents/weclouddata/my_project/spark_ML/sparkLDA/dist/sparkLDA-0.1/sparkLDA.zip")

In [5]:
sys.path = sys.path[1:]

In [6]:
sys.path

['/Users/lcx/package/spark-2.4.6-bin-hadoop2.7/python/lib/pyspark.zip',
 '/Users/lcx/package/spark-2.4.6-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip',
 '/Users/lcx/Documents/weclouddata/my_project/notebook',
 '/Users/lcx/anaconda3/envs/ML_py37/lib/python37.zip',
 '/Users/lcx/anaconda3/envs/ML_py37/lib/python3.7',
 '/Users/lcx/anaconda3/envs/ML_py37/lib/python3.7/lib-dynload',
 '',
 '/Users/lcx/anaconda3/envs/ML_py37/lib/python3.7/site-packages',
 '/Users/lcx/anaconda3/envs/ML_py37/lib/python3.7/site-packages/IPython/extensions',
 '/Users/lcx/.ipython']

In [8]:
from sparkLDA.config import n_topics, extra_for_stemmed, seedNum
from sparkLDA.utils import show_topics, evaluate
from sparkLDA.processing import preprocess_text

In [9]:
n_topics

10

In [10]:
from sparkLDA import test

In [11]:
from sparkLDA.test import hello

In [12]:
hello.say_hello()

hello
